# 그레디언트 부스팅 계열의 앙상블 모델 
- 대표 모델 : XGBoost, LightGBM
- 설치 및 사용법 숙지 필요

- 설치 여부확인

In [126]:
import pandas as pd
import numbers as np

In [127]:
import xgboost, lightgbm

print(xgboost.__version__)
print(lightgbm.__version__)

1.7.3
4.1.0


## 1. 모듈 로딩 및데이터 준비

In [128]:
## 데이터 파일 변수 선언 
# feature 중복도 있음 
feature_name_file = "../data/human_activity/features.txt"
label_name_file = "../data/human_activity/activity_labels.txt"

x_train_file = "../data/human_activity/train/X_train.txt"
y_train_file = "../data/human_activity/train/y_train.txt"

x_test_file = "../data/human_activity/test/X_test.txt"
y_test_file = "../data/human_activity/test/y_test.txt"

In [129]:
## 데이터 로딩
feature_nameDF = pd.read_csv(feature_name_file, sep="\s+", header=None, names=["col_id", "col_name"]) # , engine="python"
feature_nameDF

,col_id,col_name
0,1,tBodyAcc-mean()-X
1,2,tBodyAcc-mean()-Y
2,3,tBodyAcc-mean()-Z
3,4,tBodyAcc-std()-X
4,5,tBodyAcc-std()-Y
...,...,...
556,557,"angle(tBodyGyroMean,gravityMean)"
557,558,"angle(tBodyGyroJerkMean,gravityMean)"
558,559,"angle(X,gravityMean)"
559,560,"angle(Y,gravityMean)"


In [130]:
feature_nameDF.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 561 entries, 0 to 560
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   col_id    561 non-null    int64 
 1   col_name  561 non-null    object
dtypes: int64(1), object(1)
memory usage: 8.9+ KB


In [131]:
# 데이터 확인 -> col은 중복되면 안됨 
feature_nameDF.col_name.duplicated().sum() # 이름을 따로 붙이던가 / 인덱스를 알면 데이터랑 같이 지울 수 있음!! 

84

In [132]:
drop_index = feature_nameDF[feature_nameDF.col_name.duplicated()].index
drop_index

Index([316, 317, 318, 319, 320, 321, 322, 323, 324, 325, 326, 327, 328, 329,
       330, 331, 332, 333, 334, 335, 336, 337, 338, 339, 340, 341, 342, 343,
       395, 396, 397, 398, 399, 400, 401, 402, 403, 404, 405, 406, 407, 408,
       409, 410, 411, 412, 413, 414, 415, 416, 417, 418, 419, 420, 421, 422,
       474, 475, 476, 477, 478, 479, 480, 481, 482, 483, 484, 485, 486, 487,
       488, 489, 490, 491, 492, 493, 494, 495, 496, 497, 498, 499, 500, 501],
      dtype='int64')

In [133]:
feature_nameDF.drop(drop_index, inplace=True)
feature_nameDF=feature_nameDF.reset_index(drop=True)
feature_nameDF.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 477 entries, 0 to 476
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   col_id    477 non-null    int64 
 1   col_name  477 non-null    object
dtypes: int64(1), object(1)
memory usage: 7.6+ KB


In [134]:
x_train_file = "../data/human_activity/train/X_train.txt"
y_train_file = "../data/human_activity/train/y_train.txt"

x_test_file = "../data/human_activity/test/X_test.txt"
y_test_file = "../data/human_activity/test/y_test.txt"

In [135]:
X_trainDF = pd.read_csv(x_train_file, sep="\s+", header=None)
X_trainDF.head(3)

,0,1,2,3,4,5,6,7,8,9,...,551,552,553,554,555,556,557,558,559,560
0,0.288585,-0.020294,-0.132905,-0.995279,-0.983111,-0.913526,-0.995112,-0.983185,-0.923527,-0.934724,...,-0.074323,-0.298676,-0.710304,-0.112754,0.030400,-0.464761,-0.018446,-0.841247,0.179941,-0.058627
1,0.278419,-0.016411,-0.123520,-0.998245,-0.975300,-0.960322,-0.998807,-0.974914,-0.957686,-0.943068,...,0.158075,-0.595051,-0.861499,0.053477,-0.007435,-0.732626,0.703511,-0.844788,0.180289,-0.054317
2,0.279653,-0.019467,-0.113462,-0.995380,-0.967187,-0.978944,-0.996520,-0.963668,-0.977469,-0.938692,...,0.414503,-0.390748,-0.760104,-0.118559,0.177899,0.100699,0.808529,-0.848933,0.180637,-0.049118


In [136]:
X_trainDF.drop(drop_index, axis="columns", inplace=True)
X_trainDF=X_trainDF.T.reset_index(drop=True).T
X_trainDF.info() ############################################################ X_train 476개

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7352 entries, 0 to 7351
Columns: 477 entries, 0 to 476
dtypes: float64(477)
memory usage: 26.8 MB


In [137]:
y_trainDF = pd.read_csv(y_train_file, sep="\s+", header=None)
y_trainDF.head(3)

,0
0,5
1,5
2,5


In [138]:
X_testDF = pd.read_csv(x_test_file, sep="\s+", header=None)
X_testDF.head(3)

,0,1,2,3,4,5,6,7,8,9,...,551,552,553,554,555,556,557,558,559,560
0,0.257178,-0.023285,-0.014654,-0.938404,-0.920091,-0.667683,-0.952501,-0.925249,-0.674302,-0.894088,...,0.071645,-0.330370,-0.705974,0.006462,0.162920,-0.825886,0.271151,-0.720009,0.276801,-0.057978
1,0.286027,-0.013163,-0.119083,-0.975415,-0.967458,-0.944958,-0.986799,-0.968401,-0.945823,-0.894088,...,-0.401189,-0.121845,-0.594944,-0.083495,0.017500,-0.434375,0.920593,-0.698091,0.281343,-0.083898
2,0.275485,-0.026050,-0.118152,-0.993819,-0.969926,-0.962748,-0.994403,-0.970735,-0.963483,-0.939260,...,0.062891,-0.190422,-0.640736,-0.034956,0.202302,0.064103,0.145068,-0.702771,0.280083,-0.079346


In [139]:
X_testDF.drop(drop_index, axis="columns", inplace=True)
X_testDF=X_testDF.T.reset_index(drop=True).T
X_testDF.info() ############################################################ X_test

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2947 entries, 0 to 2946
Columns: 477 entries, 0 to 476
dtypes: float64(477)
memory usage: 10.7 MB


In [140]:
y_testDF = pd.read_csv(y_test_file, sep="\s+", header=None)
y_testDF.head(3)

,0
0,5
1,5
2,5
